### Imports and Global Variables

In [1]:
'''
Imports + global variables
'''

import os
import time
import spotipy
import spotipy.util as util
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

from pkg.spotify import *
from pkg.config import *

username = CLIENT_USERNAME
client_id = CLIENT_ID
client_secret = CLIENT_SECRET


### Logging in to Spotify with our client ID and client secret

This is where the actual Spotipy object is created and all their playlists are downloaded.

In [2]:
SPOTIPY_OBJECT = login_to_spotify(username, client_id, client_secret)
playlists = SPOTIPY_OBJECT.user_playlists(username)


ProxyError: HTTPSConnectionPool(host='accounts.spotify.com', port=443): Max retries exceeded with url: /api/token (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response',)))

### Getting playlist data

In [ ]:
likes_playlist_data = get_playlist_data(SPOTIPY_OBJECT, username, playlists, 'likes')
print('done with likes')
dislikes_playlist_data = get_playlist_data(SPOTIPY_OBJECT, username, playlists, 'dislikes')
print('done with dislikes')

### Getting our final dataframe from playlist data

This is where the actual dataframe is created.

In [ ]:
likes_df = get_dataframe(SPOTIPY_OBJECT, likes_playlist_data)
print('done with likes')

dislikes_df = get_dataframe(SPOTIPY_OBJECT, dislikes_playlist_data)
print('done with dislikes')

data_df = pd.concat([likes_df, dislikes_df], axis=0)
print('done with data')

In [ ]:
# likes_df.to_csv('likes.csv', encoding='utf-8')
# dislikes_df.to_csv('dislikes.csv', encoding='utf-8')
data_df.to_csv('data/data.csv', encoding='utf-8')
print('done with csv file')